In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-01 14:49:59--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-01 14:49:59 (14.3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch

In [2]:
import json

In [3]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"The possibility of enrolling in a course that has already started depends on several factors, including the policies of the institution offering the course and the specific circumstances surrounding your request. Here are a few general steps you can take to explore your options:\n\n1. **Check the Course Syllabus**: Review the syllabus or course outline to understand what has been covered so far and what you might need to catch up on.\n\n2. **Contact the Instructor**: Reach out directly to the course instructor. Explain your situation and ask if late enrollment is possible. They may have the discretion to allow you to join the class.\n\n3. **Consult Academic Advising**: Schedule a meeting with your academic advisor or the registrar’s office to discuss the possibility of late enrollment and any potential implications, such as missing assignments or exams.\n\n4. **Institution Policies**: Review your institution's policies regarding late enrollment. Some schools have a formal add/drop per

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
rag(query)

'To run Kafka, follow these steps based on the CONTEXT provided:\n\n1. **For Java Kafka**:\n   In the project directory, you can run the producer or consumer using the following command in the terminal:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Replace `<jar_name>` with the actual name of your JAR file.\n\n2. **For Python Kafka**:\n   To set up the environment and run a Python producer, use the following steps:\n   - Create a virtual environment and install necessary packages:\n     ```bash\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n     On Windows, instead of `source env/bin/activate`, use:\n     ```bash\n     env\\Scripts\\activate\n     ```\n   - Run the Python files within this virtual environment.\n   - If you encounter issues with Docker, make sure all Docker images are up and running before proceeding.\n\n3. **For Permission Denied Error w

In [16]:
rag('the course has already started, can I still enroll?')


"Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. Don't leave everything for the last minute."

In [17]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}